Imports

In [2]:
import datetime
import math
import os
import random
import sys

import gdal
import numpy as np
import scipy.misc
import imageio
import tensorflow as tf
from PIL import Image
from skimage import img_as_float
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import f1_score

In [3]:
NUM_CLASSES = 6


class BatchColors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

# Initialization

In [4]:
list_params = ['input_path', 'output_path(for model, images, etc)', 'currentModelPath', 'trainingInstances',
                   'testing_instances', 'learningRate', 'weight_decay', 'batch_size', 'niter', 'reference_crop_size',
                   'reference_stride_crop',
                   'net_type[dilated_icpr_original|dilated_grsl|dilated_icpr_rate6|dilated_icpr_rate6_small|'
                   'dilated_icpr_rate6_densely]',
                   'distribution_type[single_fixed|multi_fixed|uniform|multinomial]', 'probValues',
                   'update_type [acc|loss]', 'process [training|validate_test|generate_final_maps]']

## Initializing individual Parameters


In [ ]:
dataset = "postdam"
input_path = "/home/chc/Documents/CVML/dynamic-rs-segmentation/Postdam/"
output_path = 